In [ ]:
# !pip install langchain-community langchain-openai langchain-chroma langchain langchain-huggingface 

In [1]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
import os

/home/mahajan.d/.conda/envs/ai-ehr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "./data/"
loader = DirectoryLoader(path, glob="*.txt", loader_cls=TextLoader)
docs = loader.load()

In [5]:
from config import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [6]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [7]:
from config import GOOGLE_API_KEY
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [12]:
def generate_response(retriever, query):

    try:
        llm = ChatGoogleGenerativeAI(model="gemini-pro")
        # llm = ChatOpenAI(model="gpt-4")
        # llm =chat
#         llm = HuggingFacePipeline.from_model_id(
#     model_id="epfl-llm/meditron-7b",
#     task="text-generation",
# )
        template = """
            You are a helpful Medical AI assistant. Answer the question based on the context provided. Respond only if the answer is in the context.
            If the answer is not in the context then respond, Cannot answer the question.
            Stick to the context while answering and be concise.
            context: {context}
            question: {input}
            answer:
            """
        prompt = PromptTemplate.from_template(template)
        combine_docs_chain = create_stuff_documents_chain(llm, prompt)
        retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
        response = retrieval_chain.invoke({"input": query})
        # print(response["answer"])
        return response["answer"]
    except Exception as e:
        print(f"An error occurred while generating the response: {e}")
        return None

In [13]:
questions = [
"Given the History of Present Illness, What is the relevant past medical history?",
"Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital?",
"Based on the admission medication list, what other existing or ongoing medical conditions did the patient have?",
"Given the history of present illness, admission and discharge diagnosis, What medical conditions or symptoms was this patient treated for in the hospital?",
"given the history of present illness, What was the initial treatment course?",
"Given the History of Present illness, What was the initial diagnostic work up and pertinent results?",
"Given the radiology notes, What radiology, imaging or other studies that were performed and for what inducation? Summarize the impressions from each study. Make a table of Imaging study, indication, and impression."
]

In [10]:
response = generate_response(retriever, questions[0])
print(response)

The relevant past medical history includes: 

- Bipolar  
- Hepatitis C  
- HLD  
- Diabetes, steroid-induced  
- Self-reported bullous pemphigoid, but no clinic or laboratory evidence to support this diagnosis
- COPD  
- Cholelithiasis  
- GERD  
- Aortic aneurysm  
- Osteoporosis  

PAST SURGICAL HISTORY
- s/p left knee replacement
- s/p Left total knee arthroplasty explant and placement of antibiotic spacer due to sepsis, s/p removal of spacer  
- s/p 2 hip replacements  
- Compression fractures of spine  
- Inguinal hernia  
- psoriasis

PAST PSYCHIATRIC HISTORY
-Dx: Bipolar
-Hospitalizations: ___ for depression after father died 
-___: Therapist Dr. ___ at ___. (Writes clonazepam 
and TCA for "anxiety and for sleep")


In [14]:
response = generate_response(retriever, questions[0])
print(response)

PAST MEDICAL HISTORY:  
- Bipolar  
- Hepatitis C  
- HLD  
- Diabetes, steroid-induced  
- Self-reported bullous pemphigoid, but no clinic or laboratory 
evidence to support this diagnosis
- COPD  
- Cholelithiasis  
- GERD  
- Aortic aneurysm  
- Osteoporosis


In [11]:
for question in questions:
    response = generate_response(retriever, question)
    print(question,"\n")
    print(response)
    print("\n*********************")

Given the History of Present Illness, What is the relevant past medical history? 

- Bipolar  
- Hepatitis C  
- HLD  
- Diabetes, steroid-induced  
- Self-reported bullous pemphigoid, but no clinic or laboratory evidence to support this diagnosis
- COPD  
- Cholelithiasis  
- GERD  
- Aortic aneurysm  
- Osteoporosis

*********************
Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital? 

The patient was admitted to the hospital due to leg swelling, abdominal distention, and complaints of oral ulcers. The patient was experiencing significant leg pain and had a history of pemphigoid, a skin condition. The patient also reported that his abdomen was swollen. His medical condition and symptoms, along with a primary discharge diagnosis of cellulitis of the left lower limb, prompted his admission to the hospital.

*********